In [1]:
import gensim 
from gensim.models import Word2Vec , KeyedVectors

In [ ]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

vec_king = wv['king']       

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [4]:
vec_king.shape

(300,)

In [277]:
import pandas as pd 
messages = pd.read_csv(r'C:\Learn_AI\Model\Email_Spam\SMSSpamCollection.txt' , sep = '\t' , names = ['label' , 'message']) 

In [333]:
messages.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [334]:
messages.shape

(5572, 2)

In [335]:
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [348]:
from nltk.corpus import stopwords
import re
corpus = []
stopwords = stopwords.words('english')
for i in range(0 , len(messages)):
    review = re.sub('[^a-zA-Z]' , ' ' , messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords]
    review = ' '.join(review)
    if review.strip():
        corpus.append(review)
    


In [349]:
len(corpus)

5564

In [350]:
from nltk import sent_tokenize 
from gensim.utils import simple_preprocess

In [356]:
words = []
for sent in corpus : 
    sent_token = sent_tokenize(sent)
    for sent in sent_token : 
        processed_sent = simple_preprocess(sent)
        if processed_sent:
            words.append(processed_sent)
        

In [ ]:
len(words)

5559

In [358]:
import gensim

In [359]:
model= gensim.models.Word2Vec(words , min_count= 1)
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'day',
 'ok',
 'free',
 'know',
 'come',
 'like',
 'good',
 'time',
 'got',
 'love',
 'text',
 'want',
 'send',
 'need',
 'one',
 'txt',
 'today',
 'going',
 'stop',
 'home',
 'lor',
 'sorry',
 'see',
 'still',
 'mobile',
 'take',
 'back',
 'da',
 'reply',
 'dont',
 'think',
 'tell',
 'week',
 'hi',
 'phone',
 'new',
 'later',
 'please',
 'pls',
 'co',
 'msg',
 'min',
 'night',
 'make',
 'dear',
 'message',
 'well',
 'say',
 'thing',
 'much',
 'great',
 'claim',
 'hope',
 'oh',
 'hey',
 'give',
 'number',
 'happy',
 'friend',
 'work',
 'wat',
 'yes',
 'way',
 'www',
 'let',
 'prize',
 'right',
 'tomorrow',
 'already',
 'tone',
 'ask',
 'win',
 'said',
 'amp',
 'cash',
 'life',
 'yeah',
 'im',
 'really',
 'babe',
 'meet',
 'find',
 'miss',
 'morning',
 'service',
 'uk',
 'last',
 'year',
 'thanks',
 'care',
 'com',
 'would',
 'anything',
 'lol',
 'nokia',
 'also',
 'every',
 'feel',
 'keep',
 'sure',
 'pick',
 'urgent',
 'sent',
 'contact',


In [360]:
model.corpus_count

5559

In [361]:
model.wv.similar_by_word('good')

[('like', 0.9996698498725891),
 ('got', 0.9996692538261414),
 ('come', 0.9996656179428101),
 ('back', 0.9996640086174011),
 ('tell', 0.9996623992919922),
 ('much', 0.999657154083252),
 ('keep', 0.9996411204338074),
 ('today', 0.9996395707130432),
 ('thing', 0.9996395111083984),
 ('day', 0.9996383190155029)]

In [362]:
def avg_word2vec(doc):
    if not doc:
        return None  # hoặc np.zeros(model.vector_size) nếu bạn muốn vector mặc định
    vectors = [model.wv[word] for word in doc if word in model.wv.index_to_key]
    if not vectors:
        return None
    else :
        return np.mean(vectors, axis=0)

In [364]:
from tqdm import tqdm 
import numpy as np 

In [365]:
X=[]
for i in tqdm(range(len(words))):
    X.append(avg_word2vec(words[i]))

  0%|          | 0/5559 [00:00<?, ?it/s]

100%|██████████| 5559/5559 [00:00<00:00, 7501.49it/s]


In [366]:
len(X)

5559

In [367]:
len(X)

5559

In [ ]:
# # In ra các phần tử không phải là numpy ndarray
# for i, vec in enumerate(X):
#     if not isinstance(vec, np.ndarray):
#         print(f"Phần tử X[{i}] không phải là numpy array: {vec}")


In [370]:
X_new = np.array(X)

In [371]:
X_new.shape

(5559, 100)

In [385]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [386]:
len(words[0])

14

In [ ]:
y = messages[list(map(lambda x: len(x)>0 ,corpus))]
y=pd.get_dummies(y['label'])
y=y.iloc[:,0].values

ValueError: Item wrong length 5564 instead of 5572.

In [393]:
# Lọc các phần tử trống trong corpus, chỉ giữ lại những phần tử có nội dung
valid_indices = [i for i, text in enumerate(words) if len(text.strip()) > 0]

# Lọc lại cả messages và label theo chỉ số hợp lệ
messages_filtered = messages.iloc[valid_indices]
corpus_filtered = [corpus[i] for i in valid_indices]

# Kiểm tra lại độ dài
print(len(messages_filtered), len(corpus_filtered))

# Tiến hành với dữ liệu đã lọc
y = messages_filtered['label']
y = pd.get_dummies(y)
y = y.iloc[:, 0].values
len(y)

AttributeError: 'list' object has no attribute 'strip'

In [376]:
len(y)

5564

In [80]:
y.shape

(5564,)

In [81]:
messages.shape

(5572, 2)

In [74]:
y.shape

(5572,)

In [ ]:
# [[i,j,k] for i,j,k in zip(list(map(len,corpus)),corpus, messages['message']) if i<1]

[]

In [84]:
len(X)

5564

In [98]:
df = pd.DataFrame()
for i in range(0 , len(X)):
    df = pd.DataFrame(np.array(X))